Сбор данных с помощью FlightRadarAPI

In [1]:
import math
import pandas as pd
import numpy as np
from FlightRadar24 import FlightRadar24API
fr_api = FlightRadar24API()

In [2]:
# Получаем координаты аэропорта Сочи
sochi_airport = fr_api.get_airport(code="URSS")
sochi_airport_coords = [sochi_airport.latitude, sochi_airport.longitude]
sochi_airport_coords

[43.449921, 39.956581]

In [3]:
# Функция для вычисления границ квадрата, центром которого является аэропорт Сочи
def compute_bounds(airport):
    arc_length = 111.3
    down_latitude = airport.latitude - 200/arc_length
    longitude_degree_km = arc_length * math.cos(math.radians(down_latitude))
    right_longitude = airport.longitude + 200/longitude_degree_km

    up_latitude = airport.latitude + 200/arc_length
    longitude_degree_km = arc_length * math.cos(math.radians(up_latitude))
    left_longitude = airport.longitude - 200/longitude_degree_km
    bounds = f"{up_latitude},{down_latitude},{left_longitude},{right_longitude}"
    return bounds

In [4]:
# Получаем полёты, которые обрабатываются прямо сейчас в этой области
bounds = compute_bounds(sochi_airport)
flights = fr_api.get_flights(
    bounds=bounds,
    details=True
)

In [5]:
# От полёта нужно:
# id, altitude, heading

d = {'id': [], 'altitude': [], 'heading': [], 'latitude': [], 'longitude': []}

df = pd.DataFrame(data=d)

In [6]:
# функция получения данных полёта и записи их в датафрейм
def update_data(fr_api, bounds, df):
    flights = fr_api.get_flights(
        bounds=bounds,
        details=True
    )
    for i in range(len(flights)):
        id = flights[i].id
        altitude = flights[i].altitude
        heading = flights[i].heading
        latitude = flights[i].latitude
        longitude = flights[i].longitude
        df.loc[len(df.index)] = [id, altitude, heading, latitude, longitude]

In [22]:
# вызываем функцию в цикле, крутим день
# import time
# i = 0
# while i != 2*60*24:
#    update_data(fr_api, bounds, df)
#    time.sleep(30)
#    i += 1


CloudflareError: An unexpected error has occurred. Perhaps you are making too many calls?

In [20]:
# преобразуем в csv файл
#df.to_csv('data/new_data.csv', index=False)

In [21]:
# проверяем, что вышло
df_r = pd.read_csv("data/new_data.csv")
df_r

,id,altitude,heading,latitude,longitude
0,327b1463,37975,273,41.716,37.678
1,327b1959,40000,272,41.687,38.369
2,327b0897,36000,273,41.676,38.791
3,327be891,0,180,43.442,39.928
4,327c4690,0,239,43.442,39.928
...,...,...,...,...,...
25044,327f5210,27550,269,43.682,40.615
25045,327f566f,35000,64,41.947,40.615
25046,327f63ea,33975,289,43.844,40.853
25047,327f503f,36000,246,43.941,40.920
